In [ ]:
'''!pip install kaggle --quiet

import os
os.environ['KAGGLE_CONFIG_DIR'] = "~/.kaggle"  # Update this path if needed

# I had to do this because it wasn't seeing my kaggle.json
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

!kaggle competitions download -c rsna-pneumonia-detection-challenge

import zipfile
with zipfile.ZipFile('rsna-pneumonia-detection-challenge.zip', 'r') as zip_ref:
    zip_ref.extractall('rsna_data')

import os
os.listdir('rsna_data')'''

'!pip install kaggle --quiet\n\n# Step 2: Set up the Kaggle API credentials (ensure the \'kaggle.json\' is in the correct location)\nimport os\n# Make sure to place \'kaggle.json\' in the folder ~/.kaggle/\nos.environ[\'KAGGLE_CONFIG_DIR\'] = "~/.kaggle"  # Update this path if needed\n\n# I had to do this because it wasn\'t seeing my kaggle.json\nos.environ[\'KAGGLE_USERNAME\'] = \'\'\nos.environ[\'KAGGLE_KEY\'] = \'\'\n\n!kaggle competitions download -c rsna-pneumonia-detection-challenge\n\nimport zipfile\nwith zipfile.ZipFile(\'rsna-pneumonia-detection-challenge.zip\', \'r\') as zip_ref:\n    zip_ref.extractall(\'rsna_data\')\n\nimport os\nos.listdir(\'rsna_data\')'

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras import layers, models
import cv2

data_dir = './rsna_data/stage_2_train_images'
labels_df = pd.read_csv('./rsna_data/stage_2_train_labels.csv')

def load_dicom_image(image_path):
    dicom_data = pydicom.dcmread(image_path)
    
    image = dicom_data.pixel_array
    
    if len(image.shape) == 2:
        image = np.stack([image] * 3, axis=-1)  # Convert grayscale to RGB
    
    image = cv2.resize(image, (224, 224))
    
    image = image / 255.0
    
    image = (image * 255).astype(np.uint8)
    
    return image

images = []
labels = []
for idx, row in labels_df.iterrows():
    image_path = os.path.join(data_dir, f'{row["patientId"]}.dcm')
    images.append(load_dicom_image(image_path))
    labels.append(row["Target"])

images = np.array(images)
labels = np.array(labels)


2025-04-27 20:32:02.793967: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 20:32:02.911695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745800322.966142  254264 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745800322.980750  254264 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 20:32:03.081503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

'''train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)'''
train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
validation_generator = val_datagen.flow(X_val, y_val, batch_size=32)


In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Input
inputs = Input(shape=(224, 224, 3))


x = inputs
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False
x = base_model(x, training=False)

# Custom classification head
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)


I0000 00:00:1745800441.245392  254264 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21458 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


"# 9. Build the model\nmodel = models.Sequential([\n    base_model,\n    layers.GlobalAveragePooling2D(),\n    layers.Dense(1, activation='sigmoid')  # Binary classification (pneumonia vs normal)\n])\n\n# 10. Compile the model\nmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\n\n# 11. Train the model\nmodel.fit(train_generator, epochs=10, validation_data=validation_generator)\n\n# 12. Evaluate the model\nloss, accuracy = model.evaluate(validation_generator)\nprint(f'Validation Loss: {loss}')\nprint(f'Validation Accuracy: {accuracy}')"

In [ ]:

from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
	metrics=['accuracy']
)

In [ ]:
from tensorflow.keras import callbacks
# --- TRAINING ---
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True, min_delta=0.001, monitor='accuracy', mode='max')
reduce_lr = callbacks.ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop]
)

Epoch 1/50


/home/anderson_walsh/UTK/COSC523Project2/nlp/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1745800446.582120  255037 service.cc:148] XLA service 0x7f1274050560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745800446.582259  255037 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-04-27 20:34:06.682154: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745800447.201378  255037 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-27 20:34:07.964613:

  7/756 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.5139 - loss: 0.7602

I0000 00:00:1745800450.749581  255037 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


378/756 ━━━━━━━━━━━━━━━━━━━━ 6:02 958ms/step - accuracy: 0.6445 - loss: 0.6683

2025-04-27 20:34:20.616602: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6526_0', 8 bytes spill stores, 8 bytes spill loads

2025-04-27 20:34:20.701283: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6526', 16 bytes spill stores, 16 bytes spill loads

2025-04-27 20:34:20.776576: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9997', 32 bytes spill stores, 32 bytes spill loads

2025-04-27 20:34:20.783565: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9997', 20 bytes spill stores, 24 bytes spill loads

2025-04-27 20:34:20.827466: I external/local_xla/xla/stream_

504/756 ━━━━━━━━━━━━━━━━━━━━ 3:03 728ms/step - accuracy: 0.6491 - loss: 0.6635

2025-04-27 20:34:29.332008: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1702_0', 88 bytes spill stores, 120 bytes spill loads

2025-04-27 20:34:29.674891: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1702', 220 bytes spill stores, 220 bytes spill loads

2025-04-27 20:34:35.264444: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1695_0', 8 bytes spill stores, 8 bytes spill loads

2025-04-27 20:34:35.339887: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1695', 16 bytes spill stores, 16 bytes spill loads

2025-04-27 20:34:35.427212: I external/local_xla/xla/st

756/756 ━━━━━━━━━━━━━━━━━━━━ 34s 35ms/step - accuracy: 0.6562 - loss: 0.6565 - val_accuracy: 0.6839 - val_loss: 0.6282
Epoch 2/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6887 - loss: 0.6221 - val_accuracy: 0.6839 - val_loss: 0.6242
Epoch 3/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6852 - loss: 0.6175 - val_accuracy: 0.6839 - val_loss: 0.6042
Epoch 4/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6827 - loss: 0.6018 - val_accuracy: 0.6981 - val_loss: 0.5702
Epoch 5/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.6897 - loss: 0.5755 - val_accuracy: 0.7110 - val_loss: 0.5514
Epoch 6/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.6973 - loss: 0.5674 - val_accuracy: 0.7086 - val_loss: 0.5402
Epoch 7/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.7065 - loss: 0.5587 - val_accuracy: 0.7326 - val_loss: 0.5339
Epoch 8/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.7089 - loss: 0.5523 - val_accurac

In [7]:
model.save("cnn_chest_pretune_resnet.keras")
from tensorflow.keras.models import load_model
#model = load_model("cnn_chest_pretune.keras")
base_model.trainable = True

In [8]:
classifier_head_weights = [layer.get_weights() for layer in model.layers[-8:]]


In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    #metrics=[AUC(multi_label=True)]
	metrics=['accuracy']
)

In [10]:

for i, layer in enumerate(model.layers[-8:]):
    layer.set_weights(classifier_head_weights[i])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/50


2025-04-27 20:37:55.004858: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_18208', 104 bytes spill stores, 104 bytes spill loads



456/756 ━━━━━━━━━━━━━━━━━━━━ 4:06 821ms/step - accuracy: 0.7124 - loss: 0.8462

2025-04-27 20:44:27.320545: E external/local_xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module a_inference_one_step_on_data_105407__.28411] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


756/756 ━━━━━━━━━━━━━━━━━━━━ 74s 67ms/step - accuracy: 0.7268 - loss: 0.7509 - val_accuracy: 0.4299 - val_loss: 1.0360 - learning_rate: 1.0000e-05
Epoch 2/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.8326 - loss: 0.3800 - val_accuracy: 0.7474 - val_loss: 0.5225 - learning_rate: 1.0000e-05
Epoch 3/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.8887 - loss: 0.2627 - val_accuracy: 0.8296 - val_loss: 0.4217 - learning_rate: 1.0000e-05
Epoch 4/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.9561 - loss: 0.1240 - val_accuracy: 0.8248 - val_loss: 0.4877 - learning_rate: 1.0000e-05
Epoch 5/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 35s 46ms/step - accuracy: 0.9845 - loss: 0.0490 - val_accuracy: 0.7326 - val_loss: 1.0822 - learning_rate: 1.0000e-05
Epoch 6/50
646/756 ━━━━━━━━━━━━━━━━━━━━ 1:05 592ms/step - accuracy: 0.9873 - loss: 0.0344
Epoch 6: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
756/756 ━━━━━━━━━━━━━━━━━━━━ 36s 47ms/step - accuracy: 